In [1]:
import chess
from ultralytics import YOLO
from chess_engine.move_suggester import MoveSuggester
from detection.draw_overlay import draw_overlay
from utils.helpers import get_piece_positions
from chess_engine.move_detector import detect_move
import cv2

In [2]:
model = YOLO('weights/best.pt')
cap = cv2.VideoCapture("Chess_video_example.mp4")

if not cap.isOpened():
    print("Error: Cannot open video file")
    exit()
#cap = cv2.VideoCapture(0)  # 0 means default webcam

#if not cap.isOpened():
#    print("Error: Cannot open webcam")
#    exit()

In [3]:
# Initialize components
move_suggester = MoveSuggester()
board = chess.Board()
suggested = None
prev_positions = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.5, save=False, verbose=False)
    piece_positions = get_piece_positions(results, frame.shape)

    if prev_positions and piece_positions != prev_positions:
        from_sq, to_sq, piece = detect_move(prev_positions, piece_positions)
        if from_sq and to_sq and piece:
            move_uci = from_sq + to_sq
            move = chess.Move.from_uci(move_uci)
            if move in board.legal_moves:
                board.push(move)
                suggested = move_suggester.suggest_move(board)
                print(f"Last move: {piece} {from_sq}->{to_sq} | Next suggestion: {suggested}")

    prev_positions = piece_positions
    frame = draw_overlay(frame, piece_positions, suggestion=suggested, board=board)
    cv2.imshow("Chess Tutor", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
move_suggester.quit()
cv2.destroyAllWindows()


Last move: white-pawn e2->e3 | Next suggestion: e7e5
Last move: black-knight c7->c6 | Next suggestion: d2d4
